In [1]:
import spacy

In [89]:
nlp = spacy.load("output/model-best")

c:\Users\paoma\miniconda3\lib\site-packages\spacy_transformers\layers\hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fi

In [99]:
text = "tampilkan kasir yang dari tangerang dan bekasi"

doc = nlp(text)

[(w.text, w.label_) for w in doc.ents]

[('kasir', 'SUBJECT'), ('tangerang', 'LOCATION'), ('bekasi', 'LOCATION')]

Six labels for now: subject, attribute, logic, value, location and conjunction

```sql
SELECT * FROM (subject) WHERE (attribute) (logic) '(value or location)' (conjunction) (attribute) (logic) '(value or location)'
```

Assume that all texts are in the order `(attribute) (logic) (value)` or `(location)`

In [100]:
subject = ""; attribute = ""; logic = ""; values = []; locations = []

# transfer all entities into a single array
entities = [(w.text, w.label_) for w in doc.ents]

# get the subject (assume only one)
for w in entities:
    if w[1] == "SUBJECT":
        subject = w[0]
        break

# get the very first attribute
for w in entities:
    if w[1] == "ATTRIBUTE":
        old_att = w[0]
        break

# get the very first logic
for w in entities:
    if w[1] == "LOGIC":
        old_logic = w[0]
        break

# remove bad unnecessary conjunctions
last_cat = entities[0][1]
for w in entities:
    if (last_cat != "VALUE" and w[1] == "CONJUNCTION") and (last_cat != "LOCATION" and w[1] == "CONJUNCTION"):
        entities.remove(w)

    last_cat = w[1]


# create an array that holds attribute, logic and value for a single SQL argument
arguments = []

entities.append(("", "ATTRIBUTE")) # to trigger append for last entry

for w in entities:
    if w[1] == "ATTRIBUTE":
        if old_att != w[0] or locations != []:
            if values != []:
                arguments.append([attribute, logic, values])
                attribute = ""; logic = ""; values = []
            elif locations != []:
                arguments.append([attribute, logic, locations])
                attribute = ""; logic = ""; locations = []

        attribute = w[0]

        if attribute[-3:] == "nya":
            attribute = attribute[:-3]

        old_att = attribute
    elif w[1] == "LOGIC":
        if w[0] == "kurang" or "bawah" in w[0]:
            logic = "<"
        elif w[0] == "lebih" or "atas" in w[0]:
            logic = ">"
        elif w[0] == "sama":
            logic += "="
        elif "antara" in w[0]:
            logic = "AND" 

        old_logic = w[0]
    elif w[1] == "VALUE":
        # if locations != []:
        #     arguments.append([attribute, logic, locations])
        #     attribute = ""; logic = ""; values = []

        values.append(w[0])
    elif w[1] == "LOCATION":
        if values != []:
            arguments.append([attribute, logic, values])
            attribute = ""; logic = ""; values = []

        locations.append(w[0])

        attribute = "location"
        logic = "in"
    elif w[1] == "CONJUNCTION":
        if values != []:
            arguments.append([attribute, logic, values])
            attribute = ""; logic = ""; values = []
        elif locations != []:
            arguments.append([attribute, logic, locations])
            attribute = ""; logic = ""; locations = []

        arguments.append(w[0])


# clean up empty values
for arg in arguments:
    if arg[0] == "":
        arg[0] = arguments[0][0]
    if arg[1] == "":
        arg[1] = "in"


# create sql statement
sql = f"SELECT * FROM {subject} WHERE"

for arg in arguments:
    # for arguments (list)
    if isinstance(arg, list):
        # for between arguments
        if arg[1] == "AND":
            sql += f" {arg[0]} BETWEEN '{arg[2][0]}' {arg[1]} '{arg[2][1]}'"
        # for other arguments
        else:
            vals = "("
            for val in arg[2]:
                if val.isnumeric():
                    vals += val + ","
                else:
                    vals += f"'{val}',"
            vals = vals[:-1] + ")"

            sql += f" {arg[0]} {arg[1]} {vals}"
    
    # for conjunctions (string)
    elif isinstance(arg, str):
        if arg == "dan" or arg == "tapi":
            sql += " AND"
        elif arg == "atau":
            sql += " OR"

sql += ";" # to define closure of an sql statement

print(sql)

SELECT * FROM kasir WHERE location in ('tangerang','bekasi');
